## Load data with ngrams

In [1]:
import torch
import torchtext
from torchtext.datasets import text_classification

NGRAMS = 2
BATCH_SIZE = 16

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import logging
from torchtext.utils import extract_archive, unicode_csv_reader
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets.text_classification import *
from torchtext.datasets.text_classification import _csv_iterator,_create_data_from_iterator

def _setup_datasets(dataset_name, dataset_tar, root='.data', ngrams=1, vocab=None, include_unk=False):
#     dataset_tar = download_from_url(URLS[dataset_name], root=root)
    extracted_files = extract_archive(dataset_tar)

    for fname in extracted_files:
        if fname.endswith('train.csv'):
            train_csv_path = fname
        if fname.endswith('test.csv'):
            test_csv_path = fname

    if vocab is None:
        logging.info('Building Vocab based on {}'.format(train_csv_path))
        vocab = build_vocab_from_iterator(_csv_iterator(train_csv_path, ngrams))
    else:
        if not isinstance(vocab, Vocab):
            raise TypeError("Passed vocabulary is not of type Vocab")
    logging.info('Vocab has {} entries'.format(len(vocab)))
    logging.info('Creating training data')
    train_data, train_labels = _create_data_from_iterator(
        vocab, _csv_iterator(train_csv_path, ngrams, yield_cls=True), include_unk)
    logging.info('Creating testing data')
    test_data, test_labels = _create_data_from_iterator(
        vocab, _csv_iterator(test_csv_path, ngrams, yield_cls=True), include_unk)
    if len(train_labels ^ test_labels) > 0:
        raise ValueError("Training and test labels don't match")
    return (TextClassificationDataset(vocab, train_data, train_labels),
            TextClassificationDataset(vocab, test_data, test_labels))

In [3]:
import os

if not os.path.isdir("./.data"):
    os.mkdir("./.data")
    
# train_dataset, test_dataset = text_classification.DATASETS["AG_NEWS"](
#     root="./.data", ngrams=NGRAMS, vocab=None)
train_dataset, test_dataset = _setup_datasets(dataset_tar='./.data/ag_news_csv.tar.gz',dataset_name="AG_NEWS", root='./.data', ngrams=NGRAMS)

120000lines [00:07, 16656.29lines/s]
120000lines [00:11, 10219.57lines/s]
7600lines [00:00, 11018.41lines/s]


## Define the model

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        
        self.fc = nn.Linear(embed_dim, num_class)
        
        self.init_weights()
        
    def forward(self, text, offsets):
        # Inputs: input (LongTensor), offsets (LongTensor, optional)
        # it will be treated as a concatenation of multiple bags (sequences). offsets 
        # is required to be a 1D tensor containing the starting index positions of each 
        # bag in input. Therefore, for offsets of shape (B), input will be viewed as 
        # having B bags. Empty bags (i.e., having 0-length) will have returned vectors 
        # filled by zeros.
        embedded = self.embedding(text, offsets)
        
        output = self.fc(embedded)
        
        return output
        
    def init_weights(self):  # for uniform
        init_range = 0.5

        # Fills self tensor with numbers sampled from the continuous uniform distribution:
        # P(x)= 1 / (to−from)

        self.embedding.weight.data.uniform_(-init_range, init_range)
        
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

## Initiate an instance

In [5]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUM_CLASS = len(train_dataset.get_labels())

In [6]:
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS).to(device)

## Functions used to generate batch

In [7]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch], device=device)
    
    text = [entry[1] for entry in batch]
    
    # The offsets is a tensor of delimiters to represent the beginning index of the 
    # individual sequence in the text tensor.
    offsets = [0] + [len(entry) for entry in text]
    
    text = torch.cat(text).to(device)
    offsets = torch.tensor(offsets[:-1], device=device).cumsum(dim=0)
    
    return text, offsets, label

## Define functions to train the model and evaluate results.

In [8]:
from torch.utils.data import DataLoader

def train_(sub_train_):
    # Loss and accuracy.
    train_loss = 0
    train_acc = 0
    
    # Creates a data loader.
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    
    for i, (text, offsets, cls) in enumerate(data):
        # Zeros grad.
        optimizer.zero_grad()
        
        # Foreward.
        output = model(text, offsets)
        loss = criterion(output, cls)
        
        # Backward.
        loss.backward()
        
        # Updates params.
        optimizer.step()
        
        train_loss += loss.item()
        train_acc += (output.argmax(dim=1) == cls).sum().item()

    # Adjusts the learning rate.
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def predict_(data_):
    # Loss and accuracy.
    loss = 0
    acc = 0
    
    # Creates a data loader.
    data = DataLoader(sub_valid_, batch_size=BATCH_SIZE, shuffle=False,
                      collate_fn=generate_batch)
    
    for text, offsets, cls in data:
        with torch.no_grad():
            # Foreward.
            output = model(text, offsets)
            loss = criterion(output, cls)
            
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

## Split the dataset and run the model

In [9]:
# Criterion.
criterion = nn.CrossEntropyLoss()

In [10]:
# Optimizer and scheduler.

import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=4.0)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [11]:
# Splits the train set into train set and val set.

from torch.utils.data.dataset import random_split

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

In [12]:
# Calculates time.
def time_track(since):
    total_secs = time.time() - since
    
    minutes = total_secs / 60
    seconds = total_secs % 60
    
    return f"{minutes:>2.0f} minutes, {seconds:>2.0f} seconds"

In [13]:
import time

N_EPOCHS = 10
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    # Trains and valids.
    train_loss, train_acc = train_(sub_train_)
    valid_loss, valid_acc = predict_(sub_valid_)

    print(f'Epoch: {epoch + 1}', f" | time in {time_track(start_time)}")
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc:.1%}(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc:.1%}(valid)')

Epoch: 1  | time in  0 minutes,  8 seconds
	Loss: 0.0261(train)	|	Acc: 84.8%(train)
	Loss: 0.0001(valid)	|	Acc: 89.9%(valid)
Epoch: 2  | time in  0 minutes,  8 seconds
	Loss: 0.0119(train)	|	Acc: 93.7%(train)
	Loss: 0.0001(valid)	|	Acc: 90.7%(valid)
Epoch: 3  | time in  0 minutes,  8 seconds
	Loss: 0.0069(train)	|	Acc: 96.4%(train)
	Loss: 0.0000(valid)	|	Acc: 91.7%(valid)
Epoch: 4  | time in  0 minutes,  8 seconds
	Loss: 0.0039(train)	|	Acc: 98.1%(train)
	Loss: 0.0001(valid)	|	Acc: 90.9%(valid)
Epoch: 5  | time in  0 minutes,  7 seconds
	Loss: 0.0022(train)	|	Acc: 99.0%(train)
	Loss: 0.0001(valid)	|	Acc: 91.6%(valid)
Epoch: 6  | time in  0 minutes,  8 seconds
	Loss: 0.0015(train)	|	Acc: 99.4%(train)
	Loss: 0.0001(valid)	|	Acc: 91.1%(valid)
Epoch: 7  | time in  0 minutes,  8 seconds
	Loss: 0.0010(train)	|	Acc: 99.6%(train)
	Loss: 0.0001(valid)	|	Acc: 92.0%(valid)
Epoch: 8  | time in  0 minutes,  8 seconds
	Loss: 0.0008(train)	|	Acc: 99.7%(train)
	Loss: 0.0001(valid)	|	Acc: 92.0%(valid)


## Evaluate the model with test dataset

In [14]:
print("Checking the results of test dataset...")
test_loss, test_acc = predict_(test_dataset)
print(f"\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc:.1%}(test)")

Checking the results of test dataset...
	Loss: 0.0001(test)	|	Acc: 72.5%(test)


## Test on a randon news

In [15]:
import re

from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

In [16]:
def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")

    with torch.no_grad():
        # token_list = ['here', 'we', 'are']
        # list(ngrams_iterator(token_list, 2))
        # ['here', 'here we', 'we', 'we are', 'are']
        text = torch.tensor([vocab[token] for token in ngrams_iterator(
            tokenizer(text), ngrams)], device=device)
        
        output = model(text, torch.tensor([0], device=device))
        
        return output.argmax(1).item() + 1

In [17]:
vocab = train_dataset.get_vocab()
model = model.to(device)

print(f"This is a {ag_news_label[predict(ex_text_str, model, vocab, 2)]} news")

This is a Sports news
